# Bac à sable

## Check header CSV source

In [ ]:
import os.path
import pandas as pd

### Paramètres 

In [64]:
UPLOAD_FOLDER = 'uploads'
DOWNLOAD_FOLDER = 'downloads'
EXPORT_FILENAME = 'points.scr'
ALLOWED_EXTENSIONS = {'profil': 'csv','lidar': 'txt','dxf2kml': 'dxf', 'vc_lidar':'csv'}
SOURCE_FILE = '../static/data_sample/profile.csv'

d = {}
d["profil"] = ['distance', 'bathymetry', 'mns', 'mnt', 'x', 'y']
d["lidar"] = ['distance', 'altitude', 'color_packed', 'intensity', 'classification', 'coords']
d["vc_lidar"] = ['distance', 'altitude', 'color_packed', 'intensity', 'classification', 'coords']

CSV_COLUMS = d 

In [68]:
print(type(CSV_COLUMS['profil']))

<class 'list'>


### Fonctions de contrôle

In [61]:
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def check_file_extension(file: str, extension: str) -> bool:
    return True if file.rsplit('.', 1)[1].lower() == extension else False

def check_valid_csv_header(file: str, convertion: str) -> bool:
    with open(file, encoding='utf-8-sig') as f:
        first_line = f.readline().strip('\n')

    a = first_line.replace('"', '').split(",")
    b = tuple(CSV_COLUMS[convertion])
    return True if a == b else False

def read_first_line(file: str) -> bool:
    with open(file, encoding='utf-8-sig') as f:
        first_line = f.readline().strip('\n')

    truc = first_line.replace('"', '').split(",")
    print(truc)
    print(type(CSV_COLUMS['profil']))
    print(type(truc))
    for i in CSV_COLUMS['profil']:
        print(i)



def csv_header_check(df):
    source_colums = get_csv_header(df)
    if source_colums == CSV_COLUMS['profil'] or source_colums == CSV_COLUMS['lidar']:
        return True
    else:
        return False

def get_csv_header(df):
    return df.columns

def get_csv_type_profile(df):
    if csv_header_check(df) :
        source_colums = get_csv_header(df)
        if source_colums == CSV_COLUMS['profil']:
            return 'profile'
        else:
            return 'lidar'
    else:
        return


### Tests des fonctions

#### Nom du fichier

In [4]:
print(SOURCE_FILE.split('/')[-1])

profile.csv


#### Contrôle de l'extension CSV

In [5]:
file = SOURCE_FILE
file_name = file.split('/')[-1]
if allowed_file(file_name):
    print("File type pass")

File type pass


### Retourne l'extension attendu pour chaque type de conversion

In [12]:
print(ALLOWED_EXTENSIONS['profil'])
print(ALLOWED_EXTENSIONS['lidar'])
print(ALLOWED_EXTENSIONS['dxf2kml'])
print(ALLOWED_EXTENSIONS['vc_lidar'])

csv
txt
dxf
csv


### Contrôle de l'extension selon la conversion

In [22]:
file = SOURCE_FILE
extension = ALLOWED_EXTENSIONS['profil']
if check_file_extension(file,extension):
    print("File type OK")
else:
    print("File type error")

File type OK


### Lire la première line du fichier

In [57]:
read_first_line(file)

['distance', 'bathymetry', 'mns', 'mnt', 'x', 'y']
<class 'list'>
<class 'list'>
distance
bathymetry
mns
mnt
x
y


### Contrôle si l'en-tête est celle attendue

In [65]:
print(check_valid_csv_header(file,CSV_COLUMS['profil']))

TypeError: unhashable type: 'list'

#### Chargement du fichier CSV dans le dataframe

In [6]:
df = pd.read_csv(file)

#### Récupère les noms de colonnes dans une liste

In [7]:
print(df.columns.to_list())

['distance', 'bathymetry', 'mns', 'mnt', 'x', 'y']


#### Affiche les valeurs attendues pour profil et lidar

In [49]:
print(CSV_COLUMS['lidar'])
print(type(CSV_COLUMS['lidar']))
print(CSV_COLUMS['profil'])
print(CSV_COLUMS['vc_lidar'])

['distance', 'altitude', 'color_packed', 'intensity', 'classification', 'coords']
<class 'list'>
['distance', 'bathymetry', 'mns', 'mnt', 'x', 'y']
['distance', 'altitude', 'color_packed', 'intensity', 'classification', 'coords']


#### Affiche si les colonnes du fichier sont celles attendues pour profile et lidar

In [9]:
print(csv_header_check_profile(df))
print(csv_header_check_lidar(df))

NameError: name 'csv_header_check_profile' is not defined

#### Affiche le type de profil du fichier 

In [72]:
print(get_csv_type_profile(df))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()